# Exploration des CSV bruts Olist

Ce notebook analyse en profondeur les 9 fichiers CSV du dataset Olist Brazilian E-commerce.
L'objectif est de documenter les observations qui motivent les choix de modélisation
du star schema (voir `docs/csv_to_star_schema.md`).

## 0. Setup

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from src.etl.extract import load_all_raw

warnings.filterwarnings("ignore", category=FutureWarning)
sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
raw = load_all_raw()
print(f"{len(raw)} datasets chargés : {list(raw.keys())}")

In [ ]:
# Tableau récapitulatif
summary = pd.DataFrame({
    "lignes": {k: df.shape[0] for k, df in raw.items()},
    "colonnes": {k: df.shape[1] for k, df in raw.items()},
    "mémoire (MB)": {k: round(df.memory_usage(deep=True).sum() / 1e6, 2) for k, df in raw.items()},
    "doublons exacts": {k: df.duplicated().sum() for k, df in raw.items()},
})
summary.style.format({"lignes": "{:,}", "mémoire (MB)": "{:.2f}", "doublons exacts": "{:,}"})

---
## 1. Profil de chaque dataset

Pour chacun des 9 CSV : types, stats descriptives, valeurs manquantes, doublons et valeurs uniques.

In [ ]:
def profile_dataset(name: str, df: pd.DataFrame) -> None:
    """Affiche un profil complet d'un dataset."""
    print("=" * 70)
    print(f"  {name.upper()}  ({df.shape[0]:,} lignes × {df.shape[1]} colonnes)")
    print("=" * 70)

    # Info types
    print("\n--- Types et non-null ---")
    print(df.dtypes.to_frame("dtype").join(df.notnull().sum().to_frame("non_null")))

    # Stats numériques
    num_cols = df.select_dtypes(include="number").columns
    if len(num_cols) > 0:
        print("\n--- Statistiques numériques ---")
        display(df[num_cols].describe().round(2))

    # Aperçu
    print("\n--- Aperçu (5 premières lignes) ---")
    display(df.head())

    # Valeurs uniques
    print("\n--- Valeurs uniques par colonne ---")
    print(df.nunique().to_frame("n_unique"))

    # Valeurs manquantes
    missing = df.isnull().mean() * 100
    if missing.sum() > 0:
        fig, ax = plt.subplots(figsize=(8, max(3, len(df.columns) * 0.3)))
        missing.sort_values().plot.barh(ax=ax, color="salmon")
        ax.set_xlabel("% manquant")
        ax.set_title(f"{name} — Taux de valeurs manquantes")
        plt.tight_layout()
        plt.show()
    else:
        print("\n✓ Aucune valeur manquante")
    print()

In [ ]:
for name, df in raw.items():
    profile_dataset(name, df)

---
## 2. Relations et cardinalités

### Schéma des FK entre les 9 CSV
```
geolocation.zip_code_prefix ←── customers.customer_zip_code_prefix
geolocation.zip_code_prefix ←── sellers.seller_zip_code_prefix
customers.customer_id ←──────── orders.customer_id
orders.order_id ──────────────→ order_items.order_id
orders.order_id ──────────────→ order_payments.order_id
orders.order_id ──────────────→ order_reviews.order_id
products.product_id ←────────── order_items.product_id
sellers.seller_id ←──────────── order_items.seller_id
products.product_category_name → category_translation.product_category_name
```

In [ ]:
# -- Vérification des cardinalités --

orders = raw["orders"]
customers = raw["customers"]
order_items = raw["order_items"]
order_payments = raw["order_payments"]
order_reviews = raw["order_reviews"]
products = raw["products"]
sellers = raw["sellers"]
geolocation = raw["geolocation"]
cat_trans = raw["category_translation"]

print("=== Cardinalités ===")
print(f"\norders.customer_id → customers.customer_id")
print(f"  orders uniques customer_id : {orders['customer_id'].nunique():,}")
print(f"  customers uniques          : {customers['customer_id'].nunique():,}")
print(f"  → Relation N:1 (chaque commande a 1 client)")

print(f"\norder_items.order_id → orders.order_id")
items_per_order = order_items.groupby("order_id").size()
print(f"  items par commande — moy: {items_per_order.mean():.2f}, max: {items_per_order.max()}")
print(f"  → Relation N:1 (plusieurs items par commande)")

print(f"\norder_payments.order_id → orders.order_id")
pays_per_order = order_payments.groupby("order_id").size()
print(f"  paiements par commande — moy: {pays_per_order.mean():.2f}, max: {pays_per_order.max()}")
print(f"  → Relation N:1 (plusieurs paiements possibles par commande)")

print(f"\norder_reviews.order_id → orders.order_id")
reviews_per_order = order_reviews.groupby("order_id").size()
print(f"  reviews par commande — moy: {reviews_per_order.mean():.2f}, max: {reviews_per_order.max()}")
multi_review = (reviews_per_order > 1).sum()
print(f"  commandes avec >1 review : {multi_review}")
print(f"  → Relation N:1 (normalement 1 review par commande, mais {multi_review} exceptions)")

In [ ]:
# -- Orphelins : FK sans correspondance dans la table parent --

checks = [
    ("orders.customer_id", orders["customer_id"], customers["customer_id"]),
    ("order_items.order_id", order_items["order_id"], orders["order_id"]),
    ("order_items.product_id", order_items["product_id"], products["product_id"]),
    ("order_items.seller_id", order_items["seller_id"], sellers["seller_id"]),
    ("order_payments.order_id", order_payments["order_id"], orders["order_id"]),
    ("order_reviews.order_id", order_reviews["order_id"], orders["order_id"]),
]

print("=== Vérification des orphelins ===")
for label, child_col, parent_col in checks:
    orphans = child_col[~child_col.isin(parent_col)].nunique()
    status = "✓ OK" if orphans == 0 else f"⚠ {orphans} orphelins"
    print(f"  {label:35s} → {status}")

In [ ]:
# -- Distribution du nombre de lignes enfant par parent --

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

items_per_order.plot.hist(bins=range(1, items_per_order.max() + 2), ax=axes[0], color="steelblue", edgecolor="white")
axes[0].set_title("Items par commande")
axes[0].set_xlabel("Nombre d'items")

pays_per_order.plot.hist(bins=range(1, pays_per_order.max() + 2), ax=axes[1], color="coral", edgecolor="white")
axes[1].set_title("Paiements par commande")
axes[1].set_xlabel("Nombre de paiements")

reviews_per_order.plot.hist(bins=range(1, reviews_per_order.max() + 2), ax=axes[2], color="mediumseagreen", edgecolor="white")
axes[2].set_title("Reviews par commande")
axes[2].set_xlabel("Nombre de reviews")

for ax in axes:
    ax.set_ylabel("Nombre de commandes")
plt.tight_layout()
plt.show()

---
## 3. Analyses approfondies par dataset

### 3.1 geolocation — Dédoublonnage

Le CSV contient ~1M de lignes pour ~19k zip_code_prefix uniques.
Plusieurs lignes par zip avec des coordonnées et noms de ville variables.
→ Justifie l'agrégation médiane (coordonnées) + mode (ville/état).

In [ ]:
geo = raw["geolocation"]
print(f"Lignes totales : {len(geo):,}")
print(f"zip_code_prefix uniques : {geo['geolocation_zip_code_prefix'].nunique():,}")

# Nombre de lignes par zip
rows_per_zip = geo.groupby("geolocation_zip_code_prefix").size()
print(f"\nLignes par zip — moy: {rows_per_zip.mean():.1f}, médiane: {rows_per_zip.median():.0f}, max: {rows_per_zip.max()}")

fig, ax = plt.subplots(figsize=(10, 4))
rows_per_zip.clip(upper=100).plot.hist(bins=50, ax=ax, color="steelblue", edgecolor="white")
ax.set_title("Distribution du nombre de lignes par zip_code_prefix (clippé à 100)")
ax.set_xlabel("Nombre de lignes")
ax.set_ylabel("Nombre de zips")
plt.tight_layout()
plt.show()

In [ ]:
# Variabilité des coordonnées pour un même zip
coord_variability = geo.groupby("geolocation_zip_code_prefix").agg(
    lat_std=("geolocation_lat", "std"),
    lng_std=("geolocation_lng", "std"),
).dropna()

print(f"Écart-type des coordonnées par zip (sur {len(coord_variability):,} zips avec >1 ligne) :")
display(coord_variability.describe().round(4))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
coord_variability["lat_std"].clip(upper=0.1).plot.hist(bins=50, ax=axes[0], color="steelblue", edgecolor="white")
axes[0].set_title("Écart-type latitude par zip (clippé à 0.1)")
coord_variability["lng_std"].clip(upper=0.1).plot.hist(bins=50, ax=axes[1], color="coral", edgecolor="white")
axes[1].set_title("Écart-type longitude par zip (clippé à 0.1)")
plt.tight_layout()
plt.show()

In [ ]:
# Variabilité des noms de ville/état pour un même zip
city_per_zip = geo.groupby("geolocation_zip_code_prefix")["geolocation_city"].nunique()
state_per_zip = geo.groupby("geolocation_zip_code_prefix")["geolocation_state"].nunique()

print(f"Zips avec >1 nom de ville distinct : {(city_per_zip > 1).sum()}")
print(f"Zips avec >1 état distinct         : {(state_per_zip > 1).sum()}")
print("\n→ La variabilité des noms justifie l'utilisation du mode (valeur la plus fréquente)")
print("   plutôt que first() qui pourrait prendre une variante mal orthographiée.")

### 3.2 customers — Grain et identité

`customer_id` est unique par commande, `customer_unique_id` regroupe les achats d'un même client.

In [ ]:
cust = raw["customers"]
print(f"customer_id uniques        : {cust['customer_id'].nunique():,}")
print(f"customer_unique_id uniques : {cust['customer_unique_id'].nunique():,}")

ids_per_unique = cust.groupby("customer_unique_id")["customer_id"].nunique()
print(f"\ncustomer_id par customer_unique_id — moy: {ids_per_unique.mean():.2f}, max: {ids_per_unique.max()}")
print(f"Clients récurrents (>1 customer_id) : {(ids_per_unique > 1).sum():,} ({(ids_per_unique > 1).mean():.1%})")

fig, ax = plt.subplots(figsize=(8, 4))
ids_per_unique.value_counts().sort_index().plot.bar(ax=ax, color="steelblue")
ax.set_title("Distribution du nombre de customer_id par customer_unique_id")
ax.set_xlabel("Nombre de customer_id")
ax.set_ylabel("Nombre de customer_unique_id")
plt.tight_layout()
plt.show()

In [ ]:
# Couverture zip → geolocation
cust_zips = set(cust["customer_zip_code_prefix"].dropna().astype(str))
geo_zips = set(geo["geolocation_zip_code_prefix"].dropna().astype(str))
matched = cust_zips & geo_zips
print(f"Zips clients : {len(cust_zips):,}")
print(f"Zips géoloc   : {len(geo_zips):,}")
print(f"Zips matchés  : {len(matched):,} ({len(matched)/len(cust_zips):.1%})")

# Distribution géographique
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
cust["customer_state"].value_counts().head(10).plot.barh(ax=axes[0], color="steelblue")
axes[0].set_title("Top 10 états (clients)")
axes[0].invert_yaxis()
cust["customer_city"].value_counts().head(15).plot.barh(ax=axes[1], color="coral")
axes[1].set_title("Top 15 villes (clients)")
axes[1].invert_yaxis()
plt.tight_layout()
plt.show()

### 3.3 sellers — Grain et géographie

In [ ]:
sell = raw["sellers"]
print(f"seller_id uniques : {sell['seller_id'].nunique():,}")
print(f"Lignes totales    : {len(sell):,}")
print(f"→ {'Unicité vérifiée ✓' if sell['seller_id'].nunique() == len(sell) else '⚠ Doublons détectés'}")

# Couverture zip → geolocation
sell_zips = set(sell["seller_zip_code_prefix"].dropna().astype(str))
matched_s = sell_zips & geo_zips
print(f"\nZips vendeurs matchés dans géoloc : {len(matched_s):,}/{len(sell_zips):,} ({len(matched_s)/len(sell_zips):.1%})")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sell["seller_state"].value_counts().head(10).plot.barh(ax=axes[0], color="steelblue")
axes[0].set_title("Top 10 états (vendeurs)")
axes[0].invert_yaxis()
sell["seller_city"].value_counts().head(15).plot.barh(ax=axes[1], color="coral")
axes[1].set_title("Top 15 villes (vendeurs)")
axes[1].invert_yaxis()
plt.tight_layout()
plt.show()

### 3.4 orders — Temporalité et statuts

In [ ]:
ord_df = raw["orders"].copy()

# Distribution des statuts
print("=== Statuts ===")
status_counts = ord_df["order_status"].value_counts()
print(status_counts.to_frame("count").assign(pct=lambda x: (x["count"] / x["count"].sum() * 100).round(2)))

fig, ax = plt.subplots(figsize=(8, 4))
status_counts.plot.bar(ax=ax, color="steelblue")
ax.set_title("Distribution des statuts de commande")
ax.set_ylabel("Nombre de commandes")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# Plage temporelle et volume mensuel
ts_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
for col in ts_cols:
    ord_df[col] = pd.to_datetime(ord_df[col], errors="coerce")

print("=== Plage temporelle ===")
for col in ts_cols:
    non_null = ord_df[col].dropna()
    if len(non_null) > 0:
        print(f"  {col:45s} : {non_null.min().date()} → {non_null.max().date()} (null: {ord_df[col].isnull().sum():,})")

# Volume par mois
monthly = ord_df.set_index("order_purchase_timestamp").resample("ME").size()
fig, ax = plt.subplots(figsize=(12, 4))
monthly.plot(ax=ax, marker="o", color="steelblue")
ax.set_title("Volume de commandes par mois")
ax.set_ylabel("Nombre de commandes")
ax.set_xlabel("")
plt.tight_layout()
plt.show()

In [ ]:
# Timestamps NULL — lien avec le statut
print("=== Timestamps NULL par statut ===")
for col in ts_cols[1:]:
    null_by_status = ord_df[ord_df[col].isnull()]["order_status"].value_counts()
    if len(null_by_status) > 0:
        print(f"\n{col} — NULL ({ord_df[col].isnull().sum():,} lignes) :")
        print(null_by_status.to_string())

print("\n→ delivered_customer_date est NULL pour les commandes non livrées (shipped, canceled, etc.)")
print("  C'est cohérent : pas de date de livraison si la commande n'est pas arrivée.")

### 3.5 order_items — Grain article

In [ ]:
items = raw["order_items"]

# Items par commande
items_per = items.groupby("order_id").size()
print(f"Items par commande — moy: {items_per.mean():.2f}, médiane: {items_per.median():.0f}, max: {items_per.max()}")
print(f"Commandes mono-item : {(items_per == 1).sum():,} ({(items_per == 1).mean():.1%})")

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

items_per.clip(upper=10).plot.hist(bins=range(1, 12), ax=axes[0], color="steelblue", edgecolor="white")
axes[0].set_title("Items par commande (clippé à 10)")
axes[0].set_xlabel("Nombre d'items")

items["price"].plot.hist(bins=50, ax=axes[1], color="coral", edgecolor="white", log=True)
axes[1].set_title("Distribution des prix (échelle log)")
axes[1].set_xlabel("Prix (BRL)")

items["freight_value"].plot.hist(bins=50, ax=axes[2], color="mediumseagreen", edgecolor="white", log=True)
axes[2].set_title("Distribution des frais de livraison (échelle log)")
axes[2].set_xlabel("Freight (BRL)")

plt.tight_layout()
plt.show()

In [ ]:
# Outliers prix et freight
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
items[["price"]].plot.box(ax=axes[0], vert=True)
axes[0].set_title("Boxplot des prix")
items[["freight_value"]].plot.box(ax=axes[1], vert=True)
axes[1].set_title("Boxplot des frais de livraison")
plt.tight_layout()
plt.show()

print(f"Prix — min: {items['price'].min():.2f}, max: {items['price'].max():.2f}")
print(f"Freight — min: {items['freight_value'].min():.2f}, max: {items['freight_value'].max():.2f}")

In [ ]:
# Vérification FK order_items → products et sellers
orphan_products = items[~items["product_id"].isin(products["product_id"])]
orphan_sellers = items[~items["seller_id"].isin(sellers["seller_id"])]
print(f"Items sans produit correspondant : {len(orphan_products)}")
print(f"Items sans vendeur correspondant : {len(orphan_sellers)}")

### 3.6 order_payments — Paiements multiples

Une commande peut avoir plusieurs lignes de paiement (carte + voucher, etc.).
→ Justifie l'agrégation `sum(payment_value)` + `mode(payment_type)` par commande.

In [ ]:
pays = raw["order_payments"]

pays_per = pays.groupby("order_id").size()
print(f"Paiements par commande — moy: {pays_per.mean():.2f}, max: {pays_per.max()}")
print(f"Commandes mono-paiement : {(pays_per == 1).sum():,} ({(pays_per == 1).mean():.1%})")

# Commandes mixant plusieurs types de paiement
types_per_order = pays.groupby("order_id")["payment_type"].nunique()
multi_type = (types_per_order > 1).sum()
print(f"Commandes avec >1 type de paiement : {multi_type:,} ({multi_type/len(types_per_order):.1%})")

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

pays["payment_type"].value_counts().plot.bar(ax=axes[0], color="steelblue")
axes[0].set_title("Répartition par type de paiement")
axes[0].set_ylabel("Nombre de lignes")
plt.sca(axes[0])
plt.xticks(rotation=45, ha="right")

pays["payment_value"].clip(upper=1000).plot.hist(bins=50, ax=axes[1], color="coral", edgecolor="white")
axes[1].set_title("Distribution payment_value (clippé à 1000 BRL)")
axes[1].set_xlabel("Payment value (BRL)")

plt.tight_layout()
plt.show()

In [ ]:
# Détail des paiements à valeur 0
zero_pays = pays[pays["payment_value"] == 0]
print(f"Paiements à valeur 0 : {len(zero_pays):,}")
if len(zero_pays) > 0:
    print(zero_pays["payment_type"].value_counts())

print("\n→ Les paiements multiples et les types mixtes justifient l'agrégation")
print("  sum(payment_value) + mode(payment_type) par commande dans fact_orders.")

### 3.7 order_reviews — Reviews multiples

La plupart des commandes ont 1 review, mais certaines en ont plusieurs.
→ Justifie le choix de garder la review la plus récente.

In [ ]:
revs = raw["order_reviews"]

revs_per = revs.groupby("order_id").size()
multi_rev = (revs_per > 1).sum()
print(f"Reviews par commande — moy: {revs_per.mean():.2f}, max: {revs_per.max()}")
print(f"Commandes avec >1 review : {multi_rev} ({multi_rev/len(revs_per):.2%})")

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

revs["review_score"].value_counts().sort_index().plot.bar(ax=axes[0], color="steelblue")
axes[0].set_title("Distribution des review_score")
axes[0].set_xlabel("Score")
axes[0].set_ylabel("Nombre de reviews")

# Taux de commentaires vides
comment_empty = revs["review_comment_message"].isnull().mean()
title_empty = revs["review_comment_title"].isnull().mean()
pd.Series({"comment_message vide": comment_empty, "comment_title vide": title_empty}).plot.bar(
    ax=axes[1], color=["coral", "salmon"]
)
axes[1].set_title("Taux de champs texte vides")
axes[1].set_ylabel("Proportion")
axes[1].set_ylim(0, 1)
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

print(f"\nCommentaires vides : {comment_empty:.1%}")
print(f"Titres vides       : {title_empty:.1%}")
print(f"\n→ Les textes sont souvent vides. Seul review_score est retenu dans fact_orders.")
print(f"  Les {multi_rev} commandes multi-reviews justifient le choix de la review la plus récente.")

### 3.8 products + category_translation

Fusion de deux CSV : `products` contient les catégories en portugais,
`category_translation` fournit la traduction anglaise.

In [ ]:
prods = raw["products"]
cat_tr = raw["category_translation"]

# Valeurs manquantes dans products
print("=== Valeurs manquantes dans products ===")
missing_prods = prods.isnull().sum()
missing_prods = missing_prods[missing_prods > 0]
print(missing_prods.to_frame("null_count").assign(pct=lambda x: (x["null_count"] / len(prods) * 100).round(2)))

# Couverture de la traduction
cats_pt = set(prods["product_category_name"].dropna().unique())
cats_translated = set(cat_tr["product_category_name"].unique())
not_translated = cats_pt - cats_translated
print(f"\n=== Couverture traduction ===")
print(f"Catégories PT dans products  : {len(cats_pt)}")
print(f"Catégories dans translation  : {len(cats_translated)}")
print(f"Catégories non traduites     : {len(not_translated)}")
if not_translated:
    print(f"  → {not_translated}")

In [ ]:
# Top 20 catégories
fig, ax = plt.subplots(figsize=(10, 6))
prods["product_category_name"].value_counts().head(20).plot.barh(ax=ax, color="steelblue")
ax.set_title("Top 20 catégories de produits")
ax.set_xlabel("Nombre de produits")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Attributs physiques : outliers et nulls
phys_cols = ["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]

print("=== Statistiques des attributs physiques ===")
display(prods[phys_cols].describe().round(1))

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, col in zip(axes, phys_cols):
    prods[col].dropna().plot.box(ax=ax)
    ax.set_title(col.replace("product_", ""))
plt.suptitle("Boxplots des attributs physiques", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Colonnes product_name_lenght / product_description_lenght
print("=== product_name_lenght & product_description_lenght ===")
for col in ["product_name_lenght", "product_description_lenght"]:
    if col in prods.columns:
        print(f"\n{col}:")
        print(f"  null: {prods[col].isnull().sum()}, moy: {prods[col].mean():.0f}, max: {prods[col].max():.0f}")

print("\n→ Ce sont des métadonnées dérivées (longueur en caractères du nom/description).")
print("  Elles n'apportent pas de valeur analytique directe → exclues de dim_products.")